In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import Adam

In [2]:
df  = pd.read_csv('ratings.csv',index_col=False)
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
df['userId'].unique().max()

610

In [4]:
df['movieId'].unique().max()

193609

In [5]:
y = np.zeros((193610,611))
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
for i in df.index:
    # print(df['movieId'][i],df['userId'][i])
    y[df['movieId'][i]][df['userId'][i]]=df['rating'][i]
y    

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 0. , ..., 2.5, 3. , 5. ],
       [0. , 0. , 0. , ..., 2. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [7]:
y = y[:int(y.shape[0]*0.4)][:]
y.shape

(77444, 611)

In [8]:
y = tf.constant(y,shape=[y.shape[0],y.shape[1]])
y

<tf.Tensor: shape=(77444, 611), dtype=float64, numpy=
array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 0. , ..., 2.5, 3. , 5. ],
       [0. , 0. , 0. , ..., 2. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])>

In [9]:
x = tf.Variable(tf.random.uniform(shape=[y.shape[0],9],minval=0, maxval=1, dtype=tf.float64, seed=10))
x = tf.Variable(tf.concat([x, tf.ones([y.shape[0], 1], tf.float64)], 1))
w = tf.Variable(tf.random.uniform(shape=[10,y.shape[1]], maxval=5, dtype=tf.float64, seed=10))
print(x.shape, w.shape)

(77444, 10) (10, 611)


In [10]:
## Calculate forward pass and cost 
def Forward_Cost(x , w, y,u):
    f =  tf.matmul(x, w)+u
    s=  tf.math.pow((f-y),2)
    # for i in range(y.shape[0]):
    #     for j in range (y.shape[1]):
    #         if(y[i][j]!=-1):
    #             s+=tf.math.pow((yhat-y),2)
    return s           
                

In [11]:
## calculate mean
# u = np.array([])
# for j in range(y.shape[1]):
#     u1 , c = 0 , 0
#     for i in range (y.shape[0]):
#              if(y[i][j] != -1):
#                     u1=u1+y[i][j]
#                     c=c+1
#     if(c!=0):
#         np.append(u,u1/c)
#     else:
#         np.append(u,0)  
#     print(u1)    
# u= u.reshape(1,y.shape[1])    
u =tf.reduce_mean(y, 1)
u =tf.reshape(u,[u.shape[0],1])
u.shape

TensorShape([77444, 1])

In [12]:
opti= Adam(learning_rate=1e-1)
alpha =0.01
iter =200
c=[]
for i in range(iter):
    with tf.GradientTape() as tape:
        cost = Forward_Cost(x,w,y,u)
        
    c.append(cost)
    grad = tape.gradient(cost,[x,w])
    # print(grad[0])
    opti.apply_gradients(zip(grad,[x,w])) # adma optimizer

KeyboardInterrupt: 